In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotPromptTemplate, FewShotChatMessagePromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ],
    model="gpt-4o-mini"
)

examples = [
    {
    "movie": "Inception",
    "answer": """
🌌🌀🧠
"""
}, 
{
    "movie": "Parasite",
    "answer": """
🏠🌧️🍜
"""
},
{
    "movie": "The Grand Budapest Hotel",
    "answer": """
🏨🎨🕰️
"""
},
{
    "movie": "The Dark Knight",
    "answer": """
🦇🃏🌆
"""
}]

example_template = """
human: Describe {movie} in three emojis?
ai: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)

example_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Do you know about {movie}?",
    input_variables=["movie"]
)

chain = example_prompt | chat
chain.invoke({
    "movie":"The Dark Knight"
})

ai: 
    Director: Christopher Nolan
    Main Cast: Christian Bale, Heath Ledger, Aaron Eckhart
    Budget: $185 million
    Box Office: $1.005 billion
    Genre: Action, Crime, Drama
    Synopsis: Batman faces his greatest challenge yet as he takes on the Joker, a criminal mastermind who wreaks havoc on Gotham City. With the help of District Attorney Harvey Dent, Batman must navigate a web of chaos and corruption to save the city from destruction.

AIMessageChunk(content='ai: \n    Director: Christopher Nolan\n    Main Cast: Christian Bale, Heath Ledger, Aaron Eckhart\n    Budget: $185 million\n    Box Office: $1.005 billion\n    Genre: Action, Crime, Drama\n    Synopsis: Batman faces his greatest challenge yet as he takes on the Joker, a criminal mastermind who wreaks havoc on Gotham City. With the help of District Attorney Harvey Dent, Batman must navigate a web of chaos and corruption to save the city from destruction.')